In [ ]:
import cv2, os

put the path of the dataset

In [ ]:
path_for_data = ''
categories = os.listdir(path_for_data)
labels = [i for i in range (len(categories))]

label_dict = dict(zip(categories, labels))
print(label_dict)
print(categories)
print(labels)

In [ ]:
img_size = 120
data = []
target = []

for category in categories:
     folder_path = os.path.join(path_for_data, category)
     img_names = os.listdir(folder_path)
     for img_name in img_names:
            img_path = os.path.join(folder_path, img_name)
                    
            img = cv2.imread(img_path)
            
            try:
                gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
                
                resized = cv2.resize(gray, (img_size, img_size))
                
                data.append(resized)
                
                target.append(label_dict[category])
            except Exception as e:
                print('Exception',e)
                
                
            
       
     
     

In [ ]:
import numpy as np
data = np.array(data)/255.0
data = np.reshape(data,(data.shape[0], img_size,img_size,1))

target = np.array(target)


from keras.utils import np_utils
new_target = np_utils.to_categorical(target)




In [ ]:
np.save('data', data)

np.save('target', new_target)

In [ ]:
import numpy as np
from keras.models import Sequential
from keras.layers import Dense,Activation,Flatten,Dropout
from keras.layers import Conv2D,MaxPooling2D
from keras.callbacks import ModelCheckpoint
from sklearn.model_selection import train_test_split

data=np.load('data.npy')
target=np.load('target.npy')

model=Sequential()

model.add(Conv2D(100,(3,3),input_shape=data.shape[1:]))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Conv2D(100,(3,3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Flatten())
model.add(Dropout(0.5))
model.add(Dense(50,activation='relu'))
model.add(Dense(2,activation='softmax'))

model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])


In [ ]:

train_data,test_data,train_target,test_target=train_test_split(data,target,test_size=0.1)

checkpoint = ModelCheckpoint(
  'model-{epoch:03d}.model',
  monitor='val_loss',
  verbose=0,
  save_best_only=True,
  mode='auto')

history=model.fit(
  train_data,
  train_target,
  epochs=20,
  callbacks=[checkpoint],
  validation_split=0.2)

print(model.evaluate(test_data,test_target))